# Generación de textos con Redes Neuronales

En este NoteBook se creará una red que pueda generar texto.  Se verá como lo hace caracter por caracter.  En el siguiente enlace se encuentra un artículo interesante sobre esto: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

Se ha organizado el proceso en "pasos" para que fácilmente se pueda utilizar con cualquier conjunto de datos

### OJO!!! Este modelo requiere una capacidad bastante alta de cómputo. De hecho no se recomienda usarlo si no se cuenta con un GPU. Recuerde que si no tiene un equipo con GPU, una alternativa conveniente es utilizar Google Colab

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
tf.__version__

## Paso 1: Los Datos

Se puede bajar cualquier texto en forma gratuita desde este enlace:   www.gutenberg.org/

Para este ejercicio se han escogido todas las obras de Shakespeare. Esta decisión se basó en dos razones:

1. Es un cuerpo enorme de texto, generalmente se recomienda que se tenga una fuente de al menos 1 millón de caracteres para lograr una generación de texto realista.

2. Shakespeare tiene un estilo muy distinctivo:  uso de espacios y líneas nuevas, formato de sonetos, indicación de personajes en la obra, etc.  Como el texto está en inglés antiguo, y está formateado en el estilo de una obra de teatro, será muy fácil ver si el modelo puede producir resultados similares.

In [ ]:
direccion_archivo = 'shakespeare.txt'

In [ ]:
texto = open(direccion_archivo, 'r').read()

In [ ]:
print(texto[:500])

Veamos otro fragmento

In [ ]:
print(texto[4500:4800])

Y otro fragmento más

In [ ]:
print(texto[140500:141500])

Nuestra red deberá poder detectar estas estructuras y características

### Ver cuáles son los caracteres únicos

In [ ]:
# Los caracteres únicos en el archivo
vocabulario = sorted(set(texto))
print(vocabulario)
len(vocabulario)  #importante tener este número en mente para trabajar la capa Dense

## Paso 2: Procesamiento de Texto

### Vectorización de Texto

Sabemos que una red neuronal no puede recibir cadenas de texto, es necesario asignar un número a cada caracter.  Se crearán dos diccionarios que puedan ir, uno de índice numérico a caracter, y el otro de caracter a índice numérico.

Esto se puede hacer bastante fácil si usamos la función enumerate()

In [ ]:
for tupla in enumerate(vocabulario):
    print(tupla)

Usando esto, ahora vamos a crear un diccionario

In [ ]:
caract_a_indice = {u:i for i, u in enumerate(vocabulario)}

In [ ]:
caract_a_indice

In [ ]:
indice_a_caract = np.array(vocabulario)

In [ ]:
indice_a_caract

Con esto fácilmente podemos ir de un caracter a su código y viceversa...por ejemplo

In [ ]:
caract_a_indice["H"]

In [ ]:
indice_a_caract[33]

In [ ]:
texto_codificado = np.array([caract_a_indice[c] for c in texto])

In [ ]:
texto_codificado

Ahora tenemos un mapeo que nos permite ir de caracteres a numérico y viceversa.

In [ ]:
muestra = texto[:500]
muestra

In [ ]:
texto_codificado[:500]

Podemos ver cuántos caracteres tiene todo el texto

In [ ]:
texto_codificado.shape

Vemos que hay aproximadamente 5.5 millones de caracteres...más que suficiente para nuestros propósitos

Podemos ver cómo se ve el texto normal y en forma codificada

In [ ]:
texto[: 500]

In [ ]:
texto_codificado[: 500]

## Paso 3: Crear Tandas

Haremos tres cosas:

* Entender qué son las secuencias de texto
* Utilizar la clase "datasets" que tiene TensorFlow para generar las tandas
* "Barajear" las tandas

En general, lo que se trata de hacer es lograr que el modelo prediga el siguiente caracter con una alta probabilidad, dada una secuencia histórica de caracteres.  

El usuario debe decidir qué tan larga va a ser esa secuencia histórica.  Debe tomarse en cuenta que si se usa una secuencia muy corta, no se tiene suficiente información (e.g. dados la letra "a", cuál es el siguiente caracter).  En el otro extremo, si se usa una secuencia muy larga, el entrenamiento será muy largo y lo más probable es que haya un *sobre ajuste* a caracteres secuenciales que son irrelevantes a caracteres más lejanos.

Si bien no hay una selección de longitud de secuencia correcta, es importante considerar al texto mismo, qué tan largas son las frases normales que tiene, y una idea razonable sobre qué caracteres/palabras son relevantes entre sí.

In [ ]:
print(texto[:500])

In [ ]:
linea = "From fairest creatures we desire increase"

In [ ]:
len(linea)

In [ ]:
parte_estrofa = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [ ]:
len(parte_estrofa)

### Secuencias de entrenamiento

El texto actual será la secuencia de texto desplazado hacia adelante en un caracter. Por ejemplo:


Secuencia Entrante: "Hola mi nom"

Secuencia Saliente: "ola mi nomb"


Se puede usar la clase `tf.data.Dataset.from_tensor_slices` para  convertir un vector de texto a un flujo de indices de caracteres.

In [ ]:
# Viendo que una línea es de aprox 40 caracteres y que Shakespeare
#   utiliza una rima, mas o menos, a cada 3 líneas, seleccionamos:

long_secuencia = 120

In [ ]:
num_total_secuencias = len(texto)//(long_secuencia + 1)

In [ ]:
num_total_secuencias

In [ ]:
# Crear las secuencias de entrenamiento
conjunto_caract = tf.data.Dataset.from_tensor_slices(texto_codificado)
type(conjunto_caract)

Como siempre, para ver qué métodos hay disponibles para este tipo de datos, podemos escribir:

**conjunto_caract.** y luego dar un "tab"

usaremos el método **batch()**

Para ver cómo funciona, lo probaremos con una porción del texto total.  Para esto podemos usar el método **take()**


In [ ]:
for i in conjunto_caract.take(500):        # Toma un grupo de a lo sumo 500 elementos
     print(indice_a_caract[i.numpy()])     # Parar poder imprimirlos, hay que convertirlos a numpy y convertir a caracteres

El método de tandas **batch()** convierte los elementos individuales a secuencias que se pueden alimentar en tandas.  Le pasamos long_secuencia + 1 (debido a que la indización empieza en cero).  

Otro parámetro que tiene el método **batch()** es *drop_remainder*.  Este es un parámetro opcional, y es un escalar `tf.Tensor` de tipo `tf.bool`, que indica si la última secuencia debe ser "botada" en caso tenga menos de long_secuencia elementos. El valor default es 'False' o sea no botar la tanda menor.

Como nuestra secuencias son de 120 caracteres, es posible que en cada tanda quede un residuo de entre 1 y 119 caracteres.  Comparado a los casi 5 millones de caracteres de nuestro texto completo, esto es insignificante por lo que vamos a decirle que bote los residuos.


In [ ]:
secuencias = conjunto_caract.batch(long_secuencia + 1,
                                   drop_remainder = True)

Ahora que ya se tienen las secuencias, se ejecutarán los siguientes pasos para crear las secuencias de texto meta:

1. Obtener la secuencia de texto entrante
2. Asignar la secuencia de texto meta como la secuencia de texto entrante, desplazada por un paso hacia adelante
3. Agruparlos como una tupla (secuencia_entrante, secuencia_saliente)

In [ ]:
def crear_secuencias_meta(sec):
    texto_entrada = sec[:-1]      # Algo como "Hola mi nombr"
    texto_meta = sec[1:]          # Algo como "ola mi nombre"
    return texto_entrada, texto_meta

In [ ]:
# El conjunto de datos final que se alimentará a la red
datos = secuencias.map(crear_secuencias_meta)

Veamos el ejemplo de una secuencia

In [ ]:
for texto_entrada, texto_meta in datos.take(1):
    print(texto_entrada.numpy())
    print(''.join(indice_a_caract[texto_entrada.numpy()]))
    print('\n')
    print(texto_meta.numpy())
    # Hay espacio en blanco extra!
    print(''.join(indice_a_caract[texto_meta.numpy()]))

### Generar las tandas de entrenamiento

Ahora que se tienen las secuencias, se crearán las tandas.  Se "barajean" estas secuencias en un orden al azar, para que el modelo no se sobreajuste a cualquier sección de texto, pero que pueda generar caracteres dados cualquier texto "semilla".

In [ ]:
tamanio_tanda = 128     # Cuántas secuencias habrán en cada tanda...mejor si es múltiplo de dos

# Tamaño del espacio "Buffer" para barajear los datos con el fin
#   de que no intente barajear toda la secuencie en memoria.  En
#   vez, mantiene un  "buffer" en el cual barajea elementos

tamanio_buffer = 10000         # Este dependerá de cuánta memoria se tiene en el computador

datos = datos.shuffle(tamanio_buffer).batch(tamanio_tanda,
                                            drop_remainder = True)

In [ ]:
datos

Notar la forma, tenemos una tupla (la de entrada) de 128 secuencas de 120 caracteres, y otra que tupla (la de salida, **o meta**) de iguales dimensiones.

## Paso 4: Crear el Modelo

Se usará un modelo originalmente basado en LSTM con unas características extra, incluyendo una capa de incrustación "embedding" para empezar, y **dos** capas LSTM. Esta arquitectura de modelo se basa en [DeepMoji](https://deepmoji.mit.edu/) y la fuente original del código puede ser encontrada [aquí](https://github.com/bfelbo/DeepMoji).

Se puede utilizar cualquier combinación de capas pero este ejercio se hará con el modelo más simple que permita mostrar resultados "realistas".  En vez de las capas LSTM se usará una de GRU.

La capa de incrustación servirá como la capa de entrada.  Esencialmente, esta crea una tabla de consulta que mapea los índices numéricos de cada caracter a un vector con "dim_incrust" número de dimensiones.  Como es de imaginar, entre más grande este tamaño de incrustación, más complejo el entrenamiento. Hacer la incrustación antes de alimentar directamente al GRU, generalmente conlleva a resultados mas realistas.

In [ ]:
# Longitud del vocabulario de caracteres

long_vocab = len(vocabulario)
long_vocab

In [ ]:


# Dimensionamiento de la incrustación.  Se trata de que sea de
#    orden aproximado a long_vocab.  No es deseable que sea mucho
#    más grande ya que el incremento en dimensiones afecta el
#    tiempo de ejecución

dim_incrust = 64

# Número de unidades RNN

neuronas_rnn = 1024

Ahora se creará una función que se adapte fácilmente a variables diferentes como se ha mostrado arriba.

In [ ]:
from tensorflow.keras.models import Sequential

# se puede "jugar" con todo tipo de capas

from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, GRU

# para este ejemplo solo usaremos Dense, Embedding, GRU

### Configurar la función de pérdida

Para la pérdida se utilizará *sparse categorical crossentropy*, que se puede importar de Keras.  Se selecciona esta debido a que las etiquetas están "one hot encoded"

También se dejará  **from_logits = True**, ya que este parámetro se refiere a si las etiquetas están, o no, "one hot encoded"


In [ ]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
help(sparse_categorical_crossentropy)   # para mayor información

https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

Más abajo, al compilar el modelo, cuando indiquémos qué función de pérdida se desea usar, solo nos permite dar el nombre de la función, no nos permite pasar parámetros.  

Debido a que "sparse_categorical_crossentropy" tiene por default **from_logits = False**, se necesita una forma de cambiarlo.  Esto se hace envolviendo o poniendo un "wrapper" al método.


In [ ]:
def perdida_categ_escasa(y_real,y_pred):
  return sparse_categorical_crossentropy(y_real, y_pred,
                                         from_logits = True)

Definimos una función para crear el modelo de tal forma que, cuando se quiera probar con otros cojuntos de texto, será más fácil cambiar los parámetros

In [ ]:
def crear_modelo(tamanio_vocab, dim_incrust, neuronas_rnn,
                 tamanio_tanda):

    modelo = Sequential()
    
#    modelo.add(Embedding(tamanio_vocab, dim_incrust,                   #Versiones anteriores e Tensorflow
#                         batch_input_shape = [tamanio_tanda, None]))
    
    modelo.add(Embedding(input_dim = long_vocab,  # tamaño del vocabulario
                         output_dim = dim_incrust,  # dimensión de incrustación
                         input_length = None))  # usar None si la longitud de la secuencia es variable
    
    modelo.add(GRU(neuronas_rnn, return_sequences = True,
                   stateful = True,
                   recurrent_initializer = 'glorot_uniform'))

    # Capa Final Densa para Predecir
    modelo.add(Dense(long_vocab))

    modelo.compile(optimizer = 'adam', loss = perdida_categ_escasa)
    return modelo

In [ ]:
modelo = crear_modelo(long_vocab, dim_incrust,
                      neuronas_rnn, tamanio_tanda)

In [ ]:
modelo.summary()

## Paso 5: Entrenar el modelo

Antes de perder mucho tiempo con el modelo, se verifica que todo funcione bien.  Se le alimentará una tanda para asegurar que el modelo predice caracteres al azar, sin entrenar.

In [ ]:
for tanda_muestra_entrada, tanda_muestra_meta in datos.take(1):

  # Prededir a partir de una tanda al azar
  tanda_muestra_predicciones = modelo(tanda_muestra_entrada)

  # Desplegar las dimensiones de las predicciones
  print(tanda_muestra_predicciones.shape,
        " <=== (tamanio_tanda, long_secuencia, long_vocab)")


In [ ]:
tanda_muestra_predicciones

Este es un montón de probabilidades logaritmicas que asume para cada caracter concurrente, necesitamos algo que nos facilite ver estos resultados

In [ ]:
indices_muestreados = tf.random.categorical(tanda_muestra_predicciones[0],
                                            num_samples = 1)

In [ ]:
indices_muestreados

Reformatear para que no sea una lista de listas, sino que quede en el formato que deseamos para pasarlo a nuestra función de conversión a caracteres

In [ ]:

indices_muestreados = tf.squeeze(indices_muestreados,
                                 axis = -1).numpy()

In [ ]:
indices_muestreados

In [ ]:
print("Dado la secuencia de entrada: \n")
print("".join(indice_a_caract[tanda_muestra_entrada[0]]))
print('\n')
print("Predicciones del siguiente caracter: \n")
print("".join(indice_a_caract[indices_muestreados ]))


Lo que vemos es un montón de caracteres al azar, porque el modelo aún no ha sido entrenado.

Luego de confirmar las dimensiones, se procede a entrenar la red!

**NOTA:**

Este paso puede ser bastante tardado, aún con Google Colab.  Asumiendo que se ha guardado el modelo después de haberlo entrenado, se pueden  saltear las siguientres tres celdas de código y continuar con la siguiente celda.  Esta asume que el modelo entrenado se ha guardado en *shakespeare_gen.h5*

In [ ]:
epocas = 30

In [ ]:
modelo.fit(datos, epochs = epocas)

## Paso 6: Generar texto

Como está ahorita, el modelo espera 128 secuencias a la vez.  Se puede crear un modelo que espere un tamanio_tanda = 1, y luego cargar los pesos que se han guardado.  Luego se invoca *.build()* sobre el modelo:

In [ ]:
#modelo.save('shakespeare_gen.h5')
modelo.save('shakespeare.keras')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
modelo = crear_modelo(long_vocab, dim_incrust,
                      neuronas_rnn, tamanio_tanda = 1)

modelo.build(tf.TensorShape([1, None]))

#modelo.load_weights('shakespeare_gen.h5')  # Versiones anteriores de Tensorflow
modelo.load_weights('shakespeare.keras')

#modelo.build(tf.TensorShape([1, None]))  # Con versiones anteriores de Tensorflow había que poner esta instrucción aqí


In [ ]:
modelo.summary()

In [ ]:
def generar_texto(modelo, semilla_inicial, num_caract = 500, temp = 1.0):
  '''
  modelo: Modelo entrenado para Generar Texto

  semilla_inicial: Texto en formato cadena "string" a usar como semilla
  num_caract: Número de caracteres a generar

  La idea básica de esta función es la de tomar un texto semilla,
  formatearlo para que quede en la forma correcta para nuestra red,
  luego pasar la secuencia por una iteración conforme se le vayan
  agregando los caracteres predichos.  Parecido a lo que se hace
  con RNNs y series de tiempo.
  '''

  # Número de caracteres a generar
  num_generar = num_caract

  # Vectorización del texto semilla
  eval_entrada = [caract_a_indice[s] for s in semilla_inicial]

  # Expandir para llegar al formato requerido de tanda
  eval_entrada = tf.expand_dims(eval_entrada, 0)

  # Lista vacía para acumular el texto generado
  texto_generado = []

  # La "temperatura" afecta la aleatoriedad en el texto resultante
  # El término es derivado de entropía/termodinámica.
  # La "temperatura" se utiliza para afectar la probabilidad de
  #    los siguientes caracteres.
  # Temperatura mayor == menos sorprendente/ más esperado
  # Temperatura menor == más sorprendente / menos esperado

  temperatura = temp

  # Recordar que aquí tamanio_tanda == 1
  modelo.reset_states()

  for i in range(num_generar):

      # Generar Predicciones
      predicciones = modelo(eval_entrada)

      # Eliminar la dimensión de la forma de las tandas
      predicciones = tf.squeeze(predicciones, 0)

      # Usar una distribución categórica para escoger el
      #   siguiente caracter
      predicciones = predicciones / temperatura
      id_predicho = tf.random.categorical(predicciones,
                                          num_samples = 1)[-1,0].numpy()

      # Pasar el caracter predicho para la siguiente entrada
      eval_entrada = tf.expand_dims([id_predicho], 0)

      # Transformar de vuelta a una letra
      texto_generado.append(indice_a_caract[id_predicho])

  return (semilla_inicial + ''.join(texto_generado))

In [ ]:
def generar_texto(modelo, semilla_inicial, num_caract=500, temp=1.0):
    '''
    modelo: Modelo entrenado para Generar Texto
    semilla_inicial: Texto en formato cadena "string" a usar como semilla
    num_caract: Número de caracteres a generar
    temp: Temperatura para controlar la aleatoriedad del texto generado
    
    La idea básica de esta función es la de tomar un texto semilla,
    formatearlo para que quede en la forma correcta para nuestra red,
    luego pasar la secuencia por una iteración conforme se le vayan
    agregando los caracteres predichos.  Parecido a lo que se hace
    con RNNs y series de tiempo.
    '''

    # Número de caracteres a generar
    num_generar = num_caract

    # Vectorización del texto semilla
    eval_entrada = [caract_a_indice[s] for s in semilla_inicial]

    # Expandir para llegar al formato requerido de tanda
    eval_entrada = tf.expand_dims(eval_entrada, 0)

    # Lista vacía para acumular el texto generado
    texto_generado = []

    # La "temperatura" afecta la aleatoriedad en el texto resultante
    temperatura = temp

    # Removemos la línea modelo.reset_states() ya que Sequential no tiene este método
    # Si necesitas reiniciar estados, considera usar un enfoque diferente o un modelo personalizado

    for i in range(num_generar):
        # Generar Predicciones
        predicciones = modelo(eval_entrada)

        # Eliminar la dimensión de la forma de las tandas
        predicciones = tf.squeeze(predicciones, 0)

        # Usar una distribución categórica para escoger el siguiente caracter
        predicciones = predicciones / temperatura
        id_predicho = tf.random.categorical(predicciones,
                                          num_samples=1)[-1,0].numpy()

        # Pasar el caracter predicho para la siguiente entrada
        eval_entrada = tf.expand_dims([id_predicho], 0)

        # Transformar de vuelta a una letra
        texto_generado.append(indice_a_caract[id_predicho])

    return (semilla_inicial + ''.join(texto_generado))

In [ ]:
print(generar_texto(modelo, "flower", num_caract = 1000))